In [1]:
import pandas as pd
import numpy as np

In [8]:
YK_label = pd.read_excel('../data/csv_data/YK_LABEL.xlsx')
YK_data = pd.read_csv('../data/csv_data/YK_training_ULT.csv')

In [9]:
#Looking at the shape of each dataset
print(YK_data.shape)

(69577, 1076)


In [10]:
#Function to drop columns (tests) in the raw data file which are NOT tests (i.e metadata like start time, lot number etc)
def test_only(datafile,labelfile):
    keys = list(labelfile['Name'])
    df = datafile[keys]
    return df

In [ ]:
#Applying the functions to file 
YK_dropped = test_only(YK_data,YK_label)
#Checking the shape and how the data looks like
print(YK_dropped.shape)
print(YK_dropped.head())